Hola **Osvaldo**!

Soy **Patricio Requena** 👋. Es un placer ser el revisor de tu proyecto el día de hoy!

Revisaré tu proyecto detenidamente con el objetivo de ayudarte a mejorar y perfeccionar tus habilidades. Durante mi revisión, identificaré áreas donde puedas hacer mejoras en tu código, señalando específicamente qué y cómo podrías ajustar para optimizar el rendimiento y la claridad de tu proyecto. Además, es importante para mí destacar los aspectos que has manejado excepcionalmente bien. Reconocer tus fortalezas te ayudará a entender qué técnicas y métodos están funcionando a tu favor y cómo puedes aplicarlos en futuras tareas. 

_**Recuerda que al final de este notebook encontrarás un comentario general de mi parte**_, empecemos!

Encontrarás mis comentarios dentro de cajas verdes, amarillas o rojas, ⚠️ **por favor, no muevas, modifiques o borres mis comentarios** ⚠️:


<div class="alert alert-block alert-success">
<b>Comentario del revisor</b> <a class=“tocSkip”></a>
Si todo está perfecto.
</div>

<div class="alert alert-block alert-warning">
<b>Comentario del revisor</b> <a class=“tocSkip”></a>
Si tu código está bien pero se puede mejorar o hay algún detalle que le hace falta.
</div>

<div class="alert alert-block alert-danger">
<b>Comentario del revisor</b> <a class=“tocSkip”></a>
Si de pronto hace falta algo o existe algún problema con tu código o conclusiones.
</div>

Puedes responderme de esta forma:
<div class="alert alert-block alert-info">
<b>Respuesta del estudiante</b> <a class=“tocSkip”></a>
Muchísimas gracias por las observaciones 🤗
</div>

<div class="alert alert-block alert-danger">
<b>Comentario del revisor (1ra Iteracion)</b> <a class=“tocSkip”></a>

Cómo Data Scientist debes cuidar la presentación de tus proyectos en notebooks, procura utilizar más celdas markdown y sobre todo procura siempre dejar un título general al inicio del proyecto
</div>

# Clasificación de Planes Móviles con Modelos Supervisados para la Compañía Megaline

La compañía Megaline desea incentivar la migración de sus clientes hacia planes más modernos: Smart o Ultra. Para lograrlo, se planteó el desarrollo de un modelo de clasificación que, con base en el comportamiento mensual de los usuarios, recomiende el plan más adecuado. El objetivo es alcanzar una exactitud de al menos 0.75.


1. [IMPORTACIÓN DE LIBRERIAS Y DATASET](#IMPORTACIÓN-DE-LIBRERIAS-Y-DATASET)
2. [VISTA PREVIA DEL CONJUNTO DE DATOS](#VISTA-PREVIA-DEL-CONJUNTO-DE-DATOS)
3. [ENTRENAMIENTO, VALIDACIÓN Y PRUEBA](#ENTRENAMIENTO,-VALIDACIÓN-Y-PRUEBA)
4. [EVALUACIÓN DEL MODELO](#EVALUACIÓN-DEL-MODELO)
5. [CONCLUSIÓN GENERAL](#CONCLUSIÓN-GENERAL)

# 1. IMPORTACIÓN DE LIBRERIAS Y DATASET

In [1]:
import pandas as pd
import numpy as np

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [3]:
df = pd.read_csv('/datasets/users_behavior.csv')


# 2. VISTA PREVIA DEL CONJUNTO DE DATOS

In [4]:
# Mostrar primeras filas
print("📊 Vista previa del conjunto de datos:")
display(df.head())

📊 Vista previa del conjunto de datos:


,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


In [5]:
# Info general
print("\nℹ️ Información del DataFrame:")
df.info()


ℹ️ Información del DataFrame:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


Cada fila del conjunto de datos representa el comportamiento mensual de un cliente e incluye las siguientes variables:

calls: número de llamadas realizadas.

minutes: duración total en minutos de las llamadas.

messages: número de mensajes de texto enviados.

mb_used: megabytes utilizados de Internet.

is_ultra: variable objetivo (1 = plan Ultra, 0 = plan Smart).

In [6]:
# Estadísticas descriptivas
print("\n📈 Estadísticas descriptivas:")
display(df.describe())


📈 Estadísticas descriptivas:


,calls,minutes,messages,mb_used,is_ultra
count,3214.000000,3214.000000,3214.000000,3214.000000,3214.000000
mean,63.038892,438.208787,38.281269,17207.673836,0.306472
std,33.236368,234.569872,36.148326,7570.968246,0.461100
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,40.000000,274.575000,9.000000,12491.902500,0.000000
50%,62.000000,430.600000,30.000000,16943.235000,0.000000
75%,82.000000,571.927500,57.000000,21424.700000,1.000000
max,244.000000,1632.060000,224.000000,49745.730000,1.000000


In [7]:
# Duplicados
print(f"\n🔍 Registros duplicados: {df.duplicated().sum()}")


🔍 Registros duplicados: 0


Las variables están bien distribuidas y no presentan valores nulos.

Hay valores mínimos en cero, lo que indica usuarios que no usaron llamadas, mensajes o internet en algún mes. Esto es importante al entrenar el modelo (no necesariamente es un error).

La variable is_ultra tiene una media de ~0.306, lo cual sugiere un desbalance moderado en las clases (alrededor del 30% de los usuarios usan el plan Ultra).

No se encontraron registros duplicados, lo cual garantiza que los datos no están inflados ni sesgados por repeticiones.

<div class="alert alert-block alert-success">
<b>Comentario del revisor (1ra Iteracion)</b> <a class=“tocSkip”></a>

Buen trabajo con la exploración inicial de los datos para entender con lo que trabajarás
</div>

# 3. ENTRENAMIENTO, VALIDACIÓN Y PRUEBA

In [8]:
# Separar características (X) y etiqueta (y)
X = df.drop('is_ultra', axis=1)
y = df['is_ultra']

# División en conjunto de entrenamiento, validación y prueba
X_temp, X_test, y_temp, y_test = train_test_split(X, y, test_size=0.20, random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(X_temp, y_temp, test_size=0.25, random_state=42)  # 0.25 de 0.8 = 0.2

print(f"\n📐 Tamaños de los conjuntos:\nEntrenamiento: {len(X_train)}\nValidación: {len(X_valid)}\nPrueba: {len(X_test)}")



📐 Tamaños de los conjuntos:
Entrenamiento: 1928
Validación: 643
Prueba: 643


<div class="alert alert-block alert-success">
<b>Comentario del revisor (1ra Iteracion)</b> <a class=“tocSkip”></a>

Muy bien realizada la división de los datos para evaluar correctamente los modelos
</div>

# 4. EVALUACIÓN DEL MODELO

In [9]:
def probar_modelo(modelo, nombre):
    modelo.fit(X_train, y_train)
    pred = modelo.predict(X_valid)
    acc = accuracy_score(y_valid, pred)
    print(f"{nombre}: Accuracy en validación = {acc:.4f}")
    return acc

print("\n🤖 Evaluación de modelos con parámetros por defecto:")
modelos = [
    (DecisionTreeClassifier(random_state=42), "Árbol de Decisión"),
    (RandomForestClassifier(random_state=42), "Random Forest"),
    (LogisticRegression(), "Regresión Logística")
]

resultados = {nombre: probar_modelo(modelo, nombre) for modelo, nombre in modelos}



🤖 Evaluación de modelos con parámetros por defecto:
Árbol de Decisión: Accuracy en validación = 0.7309
Random Forest: Accuracy en validación = 0.7947
Regresión Logística: Accuracy en validación = 0.7201


Se entrenaron tres modelos con hiperparámetros por defecto:

Árbol de Decisión → 0.7309 de accuracy en validación

Regresión Logística → 0.7201 de accuracy

Random Forest → 0.7947 de accuracy

El modelo con mejor desempeño fue Random Forest.

In [10]:
# Seleccionar mejor modelo
mejor_modelo_nombre = max(resultados, key=resultados.get)
print(f"\n✅ Mejor modelo: {mejor_modelo_nombre}")

# Reentrenar con entrenamiento + validación para evaluación final
X_final = pd.concat([X_train, X_valid])
y_final = pd.concat([y_train, y_valid])

if mejor_modelo_nombre == "Árbol de Decisión":
    final_model = DecisionTreeClassifier(random_state=42)
elif mejor_modelo_nombre == "Random Forest":
    final_model = RandomForestClassifier(random_state=42)
else:
    final_model = LogisticRegression()

final_model.fit(X_final, y_final)
pred_test = final_model.predict(X_test)
acc_test = accuracy_score(y_test, pred_test)

print(f"\n🧪 Exactitud en el conjunto de prueba: {acc_test:.4f}")



✅ Mejor modelo: Random Forest

🧪 Exactitud en el conjunto de prueba: 0.8103


El modelo ganador (Random Forest) se reentrenó con los conjuntos de entrenamiento y validación combinados. Luego fue probado con el conjunto de test, alcanzando una exactitud final de:

✅ Accuracy en test: 0.8103

In [11]:
# Revisar si predice razonablemente bien en extremos
print("\n🧠 Prueba de cordura (primeros 5 ejemplos de test):")
print("Predicciones:", final_model.predict(X_test.head()))
print("Reales      :", y_test.head().values)



🧠 Prueba de cordura (primeros 5 ejemplos de test):
Predicciones: [0 0 0 0 1]
Reales      : [0 0 0 1 1]


Se evaluaron las primeras 5 observaciones del conjunto de prueba para verificar que las predicciones fuesen razonables.

<div class="alert alert-block alert-danger">
<b>Comentario del revisor (1ra Iteracion)</b> <a class=“tocSkip”></a>

Tienes un muy buen avance en cuando a la parte del código, pero un notebook debe incluir también la redacción de las conclusiones de los pasos realizados y sobre todo las conclusiones finales de tu proyecto
</div>

# 5. CONCLUSIÓN GENERAL

El modelo de clasificación desarrollado cumple con el objetivo del proyecto al superar el umbral de exactitud de 0.75, alcanzando un 81.03% en el conjunto de prueba. El modelo Random Forest resultó ser el más eficaz y robusto para identificar patrones de comportamiento y recomendar el plan óptimo para los usuarios.

Esta solución puede ser implementada por Megaline como un sistema inteligente de recomendación para facilitar la transición de suscriptores a planes modernos, mejorando la segmentación y fidelización de clientes.